In [1]:
# libraries from finetuning_parameters.py
from finetuning_parameters import get_args
from future.baseline_trainer import BaselineTuner
from future.modules import ptl2classes
from future.hooks import EvaluationRecorder

from data_loader.wrap_sampler import wrap_sampler
import data_loader.task_configs as task_configs
import data_loader.data_configs as data_configs
from future.collocate_fns import task2collocate_fn

import utils.checkpoint as checkpoint
import utils.logging as logging

import torch
import random
import os

# libraries from future/base.py
from torch.utils.data import SequentialSampler, RandomSampler
from future.hooks import EvaluationRecorder
import utils.eval_meters as eval_meters
from seqeval.metrics import f1_score as f1_score_tagging
import torch

# libraries from future/baseline_trainer.py
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from future.base import BaseTrainer
from future.hooks.base_hook import HookContainer
from future.hooks import EvaluationRecorder
from torch.utils.data import RandomSampler
from collections import defaultdict, Counter
from tqdm import tqdm

# and so on..
from finetuning_baseline import init_config, init_task, init_hooks

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

2022-04-13 08:56:42.366111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/ssl/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2022-04-13 08:56:42.366139: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = get_args()
conf = parser.parse_args(args=[])

In [3]:
conf.dataset_name = 'pawsx'
conf.trn_languages = 'english'
if conf.dataset_name == 'pawsx':
#     conf.eval_languages = 'english,german'
    conf.eval_languages = 'english,german,chinese,french,japanese,korean,spanish'
elif conf.dataset_name == 'xnli':
    conf.eval_languages = 'english,arabic,bulgarian,chinese,french,german,greek,hindi,russian,spanish,swahili,thai,turkish,urdu,vietnamese'
conf.finetune_epochs = 10
conf.finetune_batch_size = 256
conf.eval_every_batch = 50
conf.override = False
conf.train_fast = False
conf.world = '0'
conf.finetune_lr = 1e-5

In [4]:
init_config(conf)
model, tokenizer, data_iter, metric_name, collocate_batch_fn = init_task(conf)

data_iter

/input/jongwooko/xlt/checkpoint_baseline/pawsx/debug/1649840230_model_task-pawsx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en-de-zh-fr-ja-ko-es


parameters: 
	override	False
	experiment	debug
	ptl	bert
	model	bert-base-multilingual-cased
	dataset_name	pawsx
	max_seq_len	128
	trn_languages	['english']
	eval_languages	['english', 'german', 'chinese', 'french', 'japanese', 'korean', 'spanish']
	finetune_epochs	10
	eval_every_batch	50
	finetune_batch_size	256
	finetune_lr	1e-05
	inference_batch_size	512
	data_path	/input/jongwooko/xlt/data/download
	checkpoint	/input/jongwooko/xlt/checkpoint_baseline
	manual_seed	3
	summary_freq	100
	time_stamp	None
	train_fast	False
	track_time	True
	world	[0]
	local_rank	-1
	supcon	False
	supcon_checkpoint	None
	train_classifier	True
	train_pooler	True
	is_finished	False
	task	pawsx
	n_sub_process	1
	time_stamp_	1649840230_model_task-pawsx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-e

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.


{'chinese': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2e9ea10>,
 'english': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2a49ed0>,
 'french': <data_loader.data_iters.SeqClsDataIter at 0x7f14f29b6790>,
 'german': <data_loader.data_iters.SeqClsDataIter at 0x7f14f28b8ed0>,
 'japanese': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2899c90>,
 'korean': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2a4dfd0>,
 'spanish': <data_loader.data_iters.SeqClsDataIter at 0x7f14b6c9e5d0>}

In [5]:
adapt_loaders = {}
for language, language_dataset in data_iter.items():
    # NOTE: the sample dataset are refered
    adapt_loaders[language] = wrap_sampler(
        trn_batch_size=conf.finetune_batch_size,
        infer_batch_size=conf.inference_batch_size,
        language=language,
        language_dataset=language_dataset,
    )
hooks = init_hooks(conf, metric_name)

[WARN] trn_egs of chinese has zero egs
[WARN] trn_egs of french has zero egs
[WARN] trn_egs of german has zero egs
[WARN] trn_egs of japanese has zero egs
[WARN] trn_egs of korean has zero egs
[WARN] trn_egs of spanish has zero egs


In [6]:
adapt_loaders

{'chinese': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2e9ea10>,
 'english': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2a49ed0>,
 'french': <data_loader.data_iters.SeqClsDataIter at 0x7f14f29b6790>,
 'german': <data_loader.data_iters.SeqClsDataIter at 0x7f14f28b8ed0>,
 'japanese': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2899c90>,
 'korean': <data_loader.data_iters.SeqClsDataIter at 0x7f14f2a4dfd0>,
 'spanish': <data_loader.data_iters.SeqClsDataIter at 0x7f14b6c9e5d0>}

In [7]:
trainer = BaselineTuner(
        conf, collocate_batch_fn=collocate_batch_fn, logger=conf.logger
    )
trainer.conf.eval_languages

2022-04-13 08:57:36	Init trainer.


['english', 'german', 'chinese', 'french', 'japanese', 'korean', 'spanish']

In [8]:
import pickle

def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)
        
def load_pickle(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [11]:
if conf.dataset_name == 'xnli':
    PATH="./checkpoint_baseline/xnli/source_epoch/1649640879_model_task-xnli_flr-3.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-12272_tlang-en_vlang-en-ar-bg-zh-fr-de-el-hi-ru-es-sw-th-tr-ur-vi/state_dicts/last_state.pt"
elif conf.dataset_name == 'pawsx':
    PATH="/input/jongwooko/xlt/checkpoint_baseline/pawsx/source_epoch/1649640688_model_task-pawsx_flr-1.0E-05_ftbs-32_ftepcs-10_sd-3_trnfast-False_evalevery-1544_tlang-en_vlang-en-de-zh-fr-ja-ko-es/state_dicts/last_state.pt"
# torch.load(PATH)
model.load_state_dict(torch.load(PATH), strict=True)

<All keys matched successfully>

In [12]:
# trainer.train
opt, model = trainer._init_model_opt(model)
trainer.model = model
trainer.model.eval()

# for epoch_index in tqdm(range(1, 1 + 1)):
    
for language in trainer.conf.eval_languages: # trn_languages
    print ("Start language-{}".format(language))
    labels = np.empty((0,))
    features = np.empty((0, 768))
    trn_iters = []
    egs = adapt_loaders[language].val_egs # trn_egs
    trn_iters.append(iter(egs))

    batches_per_epoch = max(len(ti) for ti in trn_iters)
    for batch_index in range(1, batches_per_epoch + 1):
        trn_loss = []
        for ti in trn_iters:
            try:
                batched = next(ti)
            except StopIteration:
                continue
            batched, golds, uids, _golds_tagging = trainer.collocate_batch_fn(
                batched
            )
            with torch.no_grad():
                hidden = trainer.model.get_last_hidden(**batched)
                labels = np.concatenate((labels, golds.cpu()))
                features = np.concatenate((features, hidden.cpu()), axis=0)
    
    os.makedirs('./stats/{}'.format(conf.dataset_name), exist_ok=True)
    
    output_dict = {}
    for i in np.unique(labels):
        feature = features[labels == i]
        mean = np.mean(feature, axis=0)
        cov = np.cov(feature.T)
        output_dict[str(int(i))] = [{"mean": mean, "cov": cov}]
    save_pickle('./stats/{}/{}.pkl'.format(conf.dataset_name, language), output_dict)

Initialize Optimizer and Model
Start language-english
Start language-german
Start language-chinese
Start language-french
Start language-japanese
Start language-korean
Start language-spanish


# Class-Class

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cov_sim(r1, r2):
    result = np.matmul(r1, r2)
    result /= norm(r1) * norm(r2)
    return result

base_infos = load_pickle('./stats/{}/english.pkl'.format(conf.dataset_name))
for i in list(base_infos.keys()):
    print ('Start {}'.format(i))
    base_mean = base_infos[i][0]['mean']
    base_cov = base_infos[i][0]['cov'].flatten()
    for language in trainer.conf.eval_languages:
        infos = load_pickle('./stats/{}/{}.pkl'.format(conf.dataset_name, language))
        mean = infos[i][0]['mean']
        cov = infos[i][0]['cov'].flatten()
        
        cos_sim = dot(base_mean, mean)
        cos_sim /= (norm(base_mean)*norm(mean))

        cos_sim2 = dot(base_cov, cov)
        cos_sim2 /= (norm(base_cov)*norm(cov))
        # cos_sim2 = cov_sim(base_cov, cov)
        print (language, round(cos_sim, 4), round(cos_sim2, 4))
    
    print ('')